# Advanced Pandas

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('adult.csv')
df = df.rename(columns={'hours-per-week': 'hours'})
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
df.income.unique()

array(['<=50K', '>50K'], dtype=object)

In [4]:
df.race.value_counts()

White                 41762
Black                  4685
Asian-Pac-Islander     1519
Amer-Indian-Eskimo      470
Other                   406
Name: race, dtype: int64

## Finding Average Work Hours For Each Race

In [7]:
print(df[df.race == 'White'].hours.mean())
print(df[df.race == 'Black'].hours.mean())
print(df[df.race == 'Asian-Pac-Islander'].hours.mean())
print(df[df.race == 'Amer-Indian-Eskimo'].hours.mean())
print(df[df.race == 'Other'].hours.mean())

40.66016953211053
38.59786552828175
39.884792626728114
40.272340425531915
39.20197044334975


## Finding Average Work Hours For Each Race And Income Level

In [8]:
print(df[(df.race == 'White') & (df.income == '>50K')].hours.mean())
print(df[(df.race == 'White') & (df.income == '<=50K')].hours.mean())
# ..., A VERY TEDIOUS JOB!!!
# What can we do?

45.551899688884696
38.994735997432194


## Pivot Table

In [9]:
pd.pivot_table(df, index=['race'], aggfunc='mean')

,age,capital-gain,capital-loss,educational-num,fnlwgt,hours
race,,,,,,
Amer-Indian-Eskimo,36.693617,538.976596,37.800000,9.387234,120110.985106,40.272340
Asian-Pac-Islander,37.858460,1537.266623,95.557604,10.998683,159796.946675,39.884793
Black,37.914408,588.192316,61.575454,9.491142,230485.070438,38.597866
Other,33.657635,983.224138,62.958128,8.839901,195715.689655,39.201970
White,38.824362,1124.479766,90.915857,10.130262,186894.995738,40.660170


In [11]:
pd.pivot_table(df, index=['race'], aggfunc='mean').hours
# or
pd.pivot_table(df, index=['race'], values=['hours'], aggfunc='mean')

,hours
race,
Amer-Indian-Eskimo,40.272340
Asian-Pac-Islander,39.884793
Black,38.597866
Other,39.201970
White,40.660170


In [14]:
# race and income level
pd.pivot_table(df, index=['race'], columns=['income'], values=['hours'], aggfunc='mean')


hours           
income                  <=50K       >50K
race                                    
Amer-Indian-Eskimo  39.816867  43.709091
Asian-Pac-Islander  38.012613  44.965770
Black               37.824958  44.222615
Other               38.488764  44.280000
White               38.994736  45.551900

In [15]:
# there are several different aggFuncs...

## Group By

In [24]:
inc_gen_df = df.groupby(by=['gender', 'income']).mean()
# inc_gen_df.loc[('Female',  '>50K'), 'hours']
inc_gen_df

age         fnlwgt  educational-num  capital-gain  \
gender income                                                            
Female <=50K   36.278999  185690.282604         9.824724    122.876586   
       >50K    42.219333  183989.519503        11.832109   4313.663652   
Male   <=50K   37.248548  192799.098671         9.454953    162.322673   
       >50K    44.641863  189269.827082        11.561908   3993.827586   

               capital-loss      hours  
gender income                           
Female <=50K      46.910976  35.875962  
       >50K      180.222725  40.678915  
Male   <=50K      58.746173  40.720702  
       >50K      195.902299  46.304396

In [25]:
df.groupby(by=['gender', 'income']).agg(['mean', 'count'])

age                fnlwgt        educational-num         \
                    mean  count           mean  count            mean  count   
gender income                                                                  
Female <=50K   36.278999  14423  185690.282604  14423        9.824724  14423   
       >50K    42.219333   1769  183989.519503   1769       11.832109   1769   
Male   <=50K   37.248548  22732  192799.098671  22732        9.454953  22732   
       >50K    44.641863   9918  189269.827082   9918       11.561908   9918   

              capital-gain        capital-loss             hours         
                      mean  count         mean  count       mean  count  
gender income                                                            
Female <=50K    122.876586  14423    46.910976  14423  35.875962  14423  
       >50K    4313.663652   1769   180.222725   1769  40.678915   1769  
Male   <=50K    162.322673  22732    58.746173  22732  40.720702  22732  
       >50K    3993.827586   9918   195.902299   9918  46.304396   9918

## Join

In [34]:
df1 = pd.DataFrame({
    'postcode': [31323, 456433, 23998, 38752, 23421],
    'price': [81.23, 50.22, 41.55, 55.80, 15.52]
})

df2 = pd.DataFrame({
    'postcode': [31323, 456433, 23998, 38752, 23421],
    'quality': [3, 2, 2, 5, 4],
    'address': ['asdfasdf1', 'asdasdf2', 'asdfasdf3', 'asdfasdf4', 'asdasdf5']
})
df1 = df1.set_index('postcode')
df2 = df2.set_index('postcode')

print(df1)
print('\n------------------\n')
print(df2)

          price
postcode       
31323     81.23
456433    50.22
23998     41.55
38752     55.80
23421     15.52

------------------

          quality    address
postcode                    
31323           3  asdfasdf1
456433          2   asdasdf2
23998           2  asdfasdf3
38752           5  asdfasdf4
23421           4   asdasdf5


In [38]:
df1.join(df2)

,price,quality,address
postcode,,,
31323,81.23,3,asdfasdf1
456433,50.22,2,asdasdf2
23998,41.55,2,asdfasdf3
38752,55.80,5,asdfasdf4
23421,15.52,4,asdasdf5


In [41]:
pd.concat([df1, df2], axis=0)

,price,quality,address
postcode,,,
31323,81.23,NaN,NaN
456433,50.22,NaN,NaN
23998,41.55,NaN,NaN
38752,55.80,NaN,NaN
23421,15.52,NaN,NaN
31323,NaN,3.0,asdfasdf1
456433,NaN,2.0,asdasdf2
23998,NaN,2.0,asdfasdf3
38752,NaN,5.0,asdfasdf4


In [42]:
pd.concat([df1, df2], axis=1)

,price,quality,address
postcode,,,
31323,81.23,3,asdfasdf1
456433,50.22,2,asdasdf2
23998,41.55,2,asdfasdf3
38752,55.80,5,asdfasdf4
23421,15.52,4,asdasdf5
